In [2]:
#Dual Encryption using AES and Blowfish. The key of AES is encrypted using RSA.

In [3]:
import numpy as np
import blowfish
import rsa
from Cryptodome.Cipher import AES
from PIL import Image
import cv2
import math

%matplotlib auto

Using matplotlib backend: Qt5Agg


In [4]:
image = cv2.imread('bird.jpg',1)
cv2.imshow('original', image)

In [5]:
img_str = cv2.imencode('.jpg', image)[1].tobytes()

In [6]:
if len(img_str)%8 != 0:#check for compatibility
    while len(img_str)%8 != 0:
        img_str += b'\x00'

og_img_np = np.frombuffer(img_str, dtype=np.uint8)

In [7]:
cipher_b = blowfish.Cipher(b'blowfishpasswordishere')

In [8]:
data = img_str # electronic codebook mode

data_encrypted = b"".join(cipher_b.encrypt_ecb(data))

In [9]:
img_str_np_enc_blow = np.frombuffer(data_encrypted, dtype=np.uint8)
a = len(img_str_np_enc_blow)
i = 1
while a%2 == 0:
    i = i*2
    a = a//2


img_str_np_enc_blow1 = img_str_np_enc_blow.reshape(i,len(img_str_np_enc_blow)//i)

In [10]:
enc_img_blow = Image.fromarray(img_str_np_enc_blow1, 'L')
enc_img_blow.show(title='blowfish encrypted')

In [11]:
key = b'aespasswordisnow'
cipher_a = AES.new(key, AES.MODE_EAX)
nonce = cipher_a.nonce
ciphertext_enc, tag = cipher_a.encrypt_and_digest(data_encrypted)

In [12]:
public_key, private_key = rsa.newkeys(1024)

In [13]:
sig = rsa.sign(key, private_key, 'SHA-256')
enc_key = rsa.encrypt(key, public_key)

In [14]:
img_str_np_enc_aes = np.frombuffer(ciphertext_enc, dtype=np.uint8)
a = len(img_str_np_enc_aes)
i = 1
while a%2 == 0:
    i = i*2
    a = a//2


img_str_np_enc_aes1 = img_str_np_enc_aes.reshape(i,len(img_str_np_enc_aes)//i)

In [15]:
enc_img_aes = Image.fromarray(img_str_np_enc_aes1, 'L')
enc_img_aes.show(title='aes encrypted')

In [16]:
dec_key = rsa.decrypt(enc_key, private_key)
rsa.verify(dec_key, sig, public_key)

'SHA-256'

In [17]:
cipher_a = AES.new(dec_key, AES.MODE_EAX, nonce=nonce)
plaintext_enc = cipher_a.decrypt(ciphertext_enc)
try:
    cipher_a.decrypt(ciphertext_enc)
    print("AES decryption successful")
except ValueError:
    print("Decryption failed")

AES decryption successful


In [18]:
data_decrypted = b"".join(cipher_b.decrypt_ecb(plaintext_enc))

#assert data == data_decrypted

In [19]:
img_str_np = np.frombuffer(data_decrypted, dtype=np.uint8)

In [20]:
img = cv2.imdecode(img_str_np, 1)

In [21]:
cv2.imshow('decrypted', img)

In [22]:
#img_str2 = cv2.imencode('.jpg', img)[1].tobytes()
#img_str2 = img_str2[0:len(img_str)]
#img_str_np2 = np.frombuffer(img_str2, dtype=np.uint8)

In [23]:
def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

d = psnr(image, img)
print(d)

43.22586472210333


In [24]:
cv2.PSNR(image, img)

42.722213649793986

In [25]:
MSE = np.square(np.subtract(og_img_np,img_str_np)).mean()
MSE

0.0

In [26]:
Y = np.square(np.subtract(image,img)).mean()
print("MSE:", Y)

MSE: 3.0938117793787896


In [27]:
np.corrcoef(image.ravel(), img.ravel())

array([[1.        , 0.99943421],
       [0.99943421, 1.        ]])

In [28]:
def entropy(array):
    total_entropy = 0
    for i in array:
        total_entropy += -i * math.log(i, 2)
    return total_entropy

In [29]:
arr = []
arr = abs(image - img)
arr

array([[[254,   0,   0],
        [254,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[254,   0,   0],
        [254,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       ...,

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [254,   1,   0]],

       [[  1,   0,   0],
        [  2, 255,   0],
        [  2,   0,   0],
        ...,
        [  0,   1, 255],
        [  0,   0,   0],
        [254,   0,   0]],

       [[  4, 255,   0],
        [  3,   0,   0],
        [  1,   0,   0],
        ...,
        [254,   0,   0],
        [254,   1,   0],
        [254,   1,   0]]

In [30]:
entropy(abs(arr))

TypeError: only size-1 arrays can be converted to Python scalars